# Лабораторная работа №2 (проведение исследований с алгоритмом LinearRegression)

In [78]:
from ucimlrepo import fetch_ucirepo 
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, r2_score, max_error


In [46]:
glass_dataset = fetch_ucirepo(id=9)


In [47]:
df = pd.DataFrame(data=glass_dataset.data.features)
df['mpg'] = glass_dataset.data.targets

In [48]:
target_column = "mpg"

Как мы можем видеть, в наборе данных все  переменные числовые.
Пропущенные значения присуствуют в столбце horsepower. Дополним их медианным значением


In [49]:
df["horsepower"] = df["horsepower"].fillna(df["horsepower"].median())

Разобьем выборку на обучающую и валидационную

In [50]:
X = df.drop(columns=target_column, inplace=False)
y = df[target_column]

In [51]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

## 2. Создание бейзлайна и оценка качества

### Обучение модели

Определим модель и обучим модель

In [52]:
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

Сделаем предсказания

In [53]:
train_predict = model.predict(X_train)
valid_predict = model.predict(X_valid)

### Оценка качества модели

In [54]:
# Расчет метрик для тренировочных предсказаний
train_mae = mean_absolute_error(y_train, train_predict)
train_mape = mean_absolute_percentage_error(y_train, train_predict) * 100  # Преобразуем в проценты
train_r2 = r2_score(y_train, train_predict)
train_max_error = max_error(y_train, train_predict)

# Расчет метрик для валидационных предсказаний
valid_mae = mean_absolute_error(y_valid, valid_predict)
valid_mape = mean_absolute_percentage_error(y_valid, valid_predict) * 100  # Преобразуем в проценты
valid_r2 = r2_score(y_valid, valid_predict)
valid_max_error = max_error(y_valid, valid_predict)

# Сводка метрик в таблицу для удобного отображения
metrics_df = pd.DataFrame({
    'Метрика': ['MAE', 'MAPE (%)', 'R²', 'Max Error'],
    'Тренировочная выборка': [train_mae, train_mape, train_r2, train_max_error],
    'Валидационная выборка': [valid_mae, valid_mape, valid_r2, valid_max_error]
})

In [55]:
metrics_df.head()

,Метрика,Тренировочная выборка,Валидационная выборка
0,MAE,2.598833,2.255363
1,MAPE (%),11.925671,11.440774
2,R²,0.814018,0.847530
3,Max Error,12.858829,9.338143


## 3. Улучшение бейзлайна

### Препроцессинг данных

In [56]:
df_optimized = df.copy()

In [57]:
# Обработка выбросов
def remove_outliers(data, columns, threshold=1.5):
    for col in columns:
        Q1 = data[col].quantile(0.25)
        Q3 = data[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - threshold * IQR
        upper_bound = Q3 + threshold * IQR
        data = data[(data[col] >= lower_bound) & (data[col] <= upper_bound)]
    return data

df_optimized = remove_outliers(df_optimized, df_optimized.columns)

In [58]:
X = df_optimized.drop(target_column, axis=1)
y = df_optimized[target_column]

Скалирование данных

In [59]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [60]:
new_X_train, new_X_valid, new_y_train, new_y_valid = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

Решим проблему высокой корреляции данных

In [61]:
pca = PCA(n_components=new_X_train.shape[1])
pca.fit(new_X_train)

PCA(n_components=7)

Проверим сколько компонент дают 90% дисперсии

In [62]:
pca.explained_variance_ratio_.round(2)

array([0.62, 0.17, 0.13, 0.05, 0.02, 0.01, 0.  ])

Достаточно 4-ех компонент

### Обучение модели с новыми данными

Переопределим модель

In [63]:
model = Pipeline((
    ("pca", PCA(n_components=4)),
    ("model", LinearRegression())
))

In [64]:
model.fit(new_X_train, new_y_train)

Pipeline(steps=[('pca', PCA(n_components=4)), ('model', LinearRegression())])

Сделаем предсказания

In [65]:
train_predict = model.predict(new_X_train)
valid_predict = model.predict(new_X_valid)

### Оценка качетсва модели с улучшеным бейзлайном

In [66]:
# Расчет метрик для тренировочных предсказаний
train_mae = mean_absolute_error(new_y_train, train_predict)
train_mape = mean_absolute_percentage_error(new_y_train, train_predict) * 100  # Преобразуем в проценты
train_r2 = r2_score(new_y_train, train_predict)
train_max_error = max_error(new_y_train, train_predict)

# Расчет метрик для валидационных предсказаний
valid_mae = mean_absolute_error(new_y_valid, valid_predict)
valid_mape = mean_absolute_percentage_error(new_y_valid, valid_predict) * 100  # Преобразуем в проценты
valid_r2 = r2_score(new_y_valid, valid_predict)
valid_max_error = max_error(new_y_valid, valid_predict)

# Сводка метрик в таблицу для удобного отображения
metrics_df = pd.DataFrame({
    'Метрика': ['MAE', 'MAPE (%)', 'R²', 'Max Error'],
    'Тренировочная выборка': [train_mae, train_mape, train_r2, train_max_error],
    'Валидационная выборка': [valid_mae, valid_mape, valid_r2, valid_max_error]
})

In [67]:
metrics_df.head()

,Метрика,Тренировочная выборка,Валидационная выборка
0,MAE,2.627395,2.748738
1,MAPE (%),11.367555,12.635306
2,R²,0.781299,0.776540
3,Max Error,13.135121,10.968884


### Резюме:
На качество модели повлияли, как входные данные, так и параметры самой модели:
- При помощи квантилей я удалил выбросы
- Скалировал данные StandartScaller'ом
- Избавился от высокой кореляции фичей
- Подобрал параметры моделей (кол-во соседей)

## 4. Имплементация алгоритма машинного обучения

### Реализация модели

In [68]:
class LinearRegressor:
    def __init__(self, method="direct"):
        self.beta = None
        self.method = method

    def fit(self, X, y, lr=3e-3, max_iter=100):
        X, y = self._prepare_inputs(X, y)
        X = self._add_bias(X)
        self.beta = np.random.randn(X.shape[1])
        if self.method == "direct":
            self._fit_direct(X, y)
        elif self.method == "iteract":
            self._fit_iteract(X, y, lr, max_iter)
        else:
            raise AttributeError("Нет такого метода")

    def _fit_direct(self, X: np.ndarray, y: np.ndarray):
        self.beta = np.linalg.pinv(X.T @ X) @ X.T @ y

    def _fit_iteract(self, X: np.ndarray, y: np.ndarray, lr, max_iter):
        for _ in range(max_iter):
            gradient = -2 / X.shape[0] * X.T @ (y - X @ self.beta)
            self.beta -= lr * gradient

    def predict(self, X):
        X = self._prepare_inputs(X)
        X = self._add_bias(X)
        return X @ self.beta

    @staticmethod
    def _add_bias(X: np.ndarray):
        return np.hstack((np.ones((X.shape[0], 1)), X))

    @staticmethod
    def _prepare_inputs(X, y=None):
        if isinstance(X, pd.DataFrame) or isinstance(X, pd.Series):
            X = X.values
        if isinstance(X, list):
            X = np.array(X)
        X = X.astype(float)

        if y is not None:
            if isinstance(y, pd.DataFrame) or isinstance(y, pd.Series):
                y = y.values
            if isinstance(y, list):
                y = np.array(y)
            if isinstance(y, pd.CategoricalDtype):
                y = y.astype(int)
            y = y.astype(float)
            return X, y

        return X

### Обучение 

In [69]:
model = LinearRegressor()
model.fit(X_train, y_train)

In [70]:
train_predict = model.predict(X_train)
valid_predict = model.predict(X_valid)

###  Оценка качества модели

In [71]:
# Расчет метрик для тренировочных предсказаний
train_mae = mean_absolute_error(y_train, train_predict)
train_mape = mean_absolute_percentage_error(y_train, train_predict) * 100  # Преобразуем в проценты
train_r2 = r2_score(y_train, train_predict)
train_max_error = max_error(y_train, train_predict)

# Расчет метрик для валидационных предсказаний
valid_mae = mean_absolute_error(y_valid, valid_predict)
valid_mape = mean_absolute_percentage_error(y_valid, valid_predict) * 100  # Преобразуем в проценты
valid_r2 = r2_score(y_valid, valid_predict)
valid_max_error = max_error(y_valid, valid_predict)

# Сводка метрик в таблицу для удобного отображения
metrics_df = pd.DataFrame({
    'Метрика': ['MAE', 'MAPE (%)', 'R²', 'Max Error'],
    'Тренировочная выборка': [train_mae, train_mape, train_r2, train_max_error],
    'Валидационная выборка': [valid_mae, valid_mape, valid_r2, valid_max_error]
})

In [72]:
metrics_df.head()

,Метрика,Тренировочная выборка,Валидационная выборка
0,MAE,2.598833,2.255363
1,MAPE (%),11.925671,11.440774
2,R²,0.814018,0.847530
3,Max Error,12.858829,9.338143


### Обучение с улучшенным бейзлайном

In [73]:
model = Pipeline((
    ("pca", PCA(n_components=4)),
    ("model", LinearRegressor())
))

In [74]:
model.fit(new_X_train, new_y_train)

Pipeline(steps=[('pca', PCA(n_components=4)),
                ('model',
                 <__main__.LinearRegressor object at 0x0000021DD1D0E5E0>)])

In [75]:
train_predict = model.predict(new_X_train)
valid_predict = model.predict(new_X_valid)

###  Оценка качества модели

In [76]:
# Расчет метрик для тренировочных предсказаний
train_mae = mean_absolute_error(new_y_train, train_predict)
train_mape = mean_absolute_percentage_error(new_y_train, train_predict) * 100  # Преобразуем в проценты
train_r2 = r2_score(new_y_train, train_predict)
train_max_error = max_error(new_y_train, train_predict)

# Расчет метрик для валидационных предсказаний
valid_mae = mean_absolute_error(new_y_valid, valid_predict)
valid_mape = mean_absolute_percentage_error(new_y_valid, valid_predict) * 100  # Преобразуем в проценты
valid_r2 = r2_score(new_y_valid, valid_predict)
valid_max_error = max_error(new_y_valid, valid_predict)

# Сводка метрик в таблицу для удобного отображения
metrics_df = pd.DataFrame({
    'Метрика': ['MAE', 'MAPE (%)', 'R²', 'Max Error'],
    'Тренировочная выборка': [train_mae, train_mape, train_r2, train_max_error],
    'Валидационная выборка': [valid_mae, valid_mape, valid_r2, valid_max_error]
})

In [77]:
metrics_df.head()

,Метрика,Тренировочная выборка,Валидационная выборка
0,MAE,2.627395,2.748738
1,MAPE (%),11.367555,12.635306
2,R²,0.781299,0.776540
3,Max Error,13.135121,10.968884
